In [4]:
from src.data_preprocessor import load_and_process_data, aggregate_to_yearly
import pandas as pd

In [5]:
df = load_and_process_data()

2023-11-06 22:40:20,088 - INFO - Loading the latest recent file: data/lsoa-recent.csv
2023-11-06 22:40:20,477 - INFO - Loading the latest historical file: data/lsoa-historical.csv


In [10]:
yearly_df = aggregate_to_yearly(df).sort_index(axis=1)
yearly_df

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,Borough,LSOA Code,LSOA Name,Major Category,Minor Category
0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,Barking and Dagenham,E01000006,Barking and Dagenham 016A,Arson and Criminal Damage,Arson
1,9.0,4.0,7.0,6.0,8.0,7.0,7.0,5.0,12.0,6.0,4.0,5.0,6.0,4.0,Barking and Dagenham,E01000006,Barking and Dagenham 016A,Arson and Criminal Damage,Criminal Damage
2,1.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,Barking and Dagenham,E01000006,Barking and Dagenham 016A,Burglary,Burglary Business and Community
3,13.0,17.0,15.0,12.0,10.0,10.0,7.0,12.0,15.0,13.0,5.0,4.0,3.0,2.0,Barking and Dagenham,E01000006,Barking and Dagenham 016A,Burglary,Domestic Burglary
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,Barking and Dagenham,E01000006,Barking and Dagenham 016A,Drug Offences,Drug Trafficking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Greenwich,E01033742,Greenwich 007F,Miscellaneous Crimes Against Society,"Making, Supplying or Possessing Articles for u..."
153308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Greenwich,E01033742,Greenwich 007F,Possession of Weapons,Possession of Article with Blade or Point
153309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Greenwich,E01033745,Greenwich 002I,Possession of Weapons,Other Firearm Offences
153310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,Greenwich,E01033746,Greenwich 038E,Possession of Weapons,Other Firearm Offences
